In [1]:
import numpy as np
import pandas as pd
import re
from pandas import Series, DataFrame
gtes = "gtes04feb.xls"
goi = "goi04feb.xls"
gom = "gom04feb.xls"
gef = "gef04feb.xls"
parametros = "parametros.xlsx"
cuentas=[7,10,11,12,16,17]
bdgtes = pd.read_excel('../data/'+gtes)
bdgoi = pd.read_excel('../data/'+goi)
bdgom = pd.read_excel('../data/'+gom)
bdgef = pd.read_excel('../data/'+gef)
movimientos =pd.read_excel('../data/'+parametros, sheet_name="movimientos")
bdBruto= pd.concat([bdgtes,bdgoi,bdgom,bdgef],ignore_index=True)
#bd.describe()
movimientos


WARNING *** file size (751214) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (394879) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (4014373) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (4367209) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,Concepto,Mayor2,codMov,nomMovimiento,codigoBC,codigoPropio
0,Ingreso,404,7164,YPFB GLP,Y.P.F.B. Otros (BCI5),YPFB GLP
1,Ingreso,404,7954,YPFB UREA,Y.P.F.B. Otros (BCI5),YPFB UREA
2,Ingreso,404,5968,YPFB Héroes del Chaco,Ver Glosa,Ver Glosa
3,Egreso,404,5968,YPFB Héroes del Chaco,Ver Glosa,Ver Glosa
4,Ingreso,404,6215,Empresa Metalúrgica Vinto,VINTO (BCI6),Empresa Metalúrgica Vinto
...,...,...,...,...,...,...
63,Egreso,498,6818,Dep. Sector Público,Ver Glosa,Ver Glosa
64,Egreso,25,166,Aladi,"CONVENIO CRED. RECIP.(1),(2) (BCE6)",Aladi
65,Ingreso,25,9999,Aladi,CONVENIO DE CRED. RECIP (1) (BCI11),Aladi
66,Ingreso,828,3500,Ver estos,CUENTA CORRIENTE (3) (BCI18),Ver Glosa


In [2]:
#bdBruto.columns

In [3]:
bdReducida=bdBruto.loc[:,('cve_tipo_comprob','glosa_reng','cve_debe_haber','monto_mo',
           'cod_moneda', 'cod_movimiento', 'nom_movimiento',
           'monto_mn','glosa_comprob','nro_comprob','cod_mayor')]
#bdReducida

In [4]:
dict = {'cve_tipo_comprob':'tipo','glosa_reng':'glosaRenglon','cve_debe_haber':'dh','monto_mo':'mo',
           'cod_moneda':'moneda', 'cod_movimiento':'codMov', 'nom_movimiento':'nomMov',
           'monto_mn':'mn','glosa_comprob':'glosa','nro_comprob':'comprobante','cod_mayor':'mayor'}
bdReducida.rename(columns=dict,
          inplace=True)
bdReducida.shape

(6913, 11)

In [5]:
#bd['mayor'].unique()

#bdFiltro1=bdReducida[bdReducida['mayor'].isin([7,10,11,16,17])]
bdFiltro1=bdReducida[bdReducida['mayor'].isin(cuentas)]

In [6]:
#bdFiltro1.loc[:,("comprobante","mayor")]
bdFiltro1["comprobante"].unique()
#bdFiltro1.sort_values(by="comprobante", ascending=True)

array([1996851, 1996852, 1996865, 1996906, 1996980, 1025269,   28482,
       1777767,   21859, 1025316, 1025295, 1777769, 1777343, 1777345,
       1777347, 1777480, 1777481, 1777482, 1777483, 1025283, 1025275,
       1025272, 1025273, 1777620, 1777623, 1777626, 1777629, 1777631,
       1777633, 1025289, 1025312, 1025314, 1025301, 1777765, 1025315,
       1025332, 1025334,    5545,    2178], dtype=int64)

In [7]:
#bd[bd['comprobante'].isin(bdFiltro1["comprobante"])].sort_values(by="comprobante", ascending=True).tail(60)
bd=bdReducida[bdReducida["comprobante"].isin(bdFiltro1["comprobante"])].sort_values(by=["comprobante","dh"], ascending=True)
bd = bd.reset_index(drop=True)
#bd.head(60)

In [8]:
#list(bd.loc[55,["glosa"]])

In [9]:
#bd.to_excel("ux.xlsx")

In [10]:
bd=bd.drop_duplicates()
bd = bd.reset_index(drop=True)
#bd.head(60)

In [11]:
#bd["comprobante"].unique()

In [12]:
def llenarData(concepto,a,b):
    if concepto !="Neteo":    
        print('dentro ',pares["Comprobante"][a])
        data2.append({
                 "Tipo":pares["Tipo"][a],"Comprobante":pares["Comprobante"][a],"Concepto":concepto,
                "Mayor1":pares["Mayor"][a],"Mayor2":pares["Mayor"][b],"codMov":pares["codMov"][b],
                "Movimiento":pares["Movimiento"][b],"MontoUSD":round(pares["MN"][a]/6.86,2),
                "Detalle":pares["GlosaDetalle"][b],"Glosa":pares["Glosa"][a]
                })
    else:
        data2.append({
                "Tipo":pares["Tipo"][a],"Comprobante":pares["Comprobante"][a],"Concepto":concepto,
                "Mayor1":pares["Mayor"][a],"Mayor2":pares["Mayor"][b],"codMov":pares["codMov"][b],
                "Movimiento":pares["Movimiento"][b],"MontoUSD":0, 
                "Detalle":("±", round(pares["MN"][a]/6.86,2)),"Glosa":pares["Glosa"][a]
                })

def dv(concepto,suma):
    print("largo cuando dentra al debehaber: ",len(aux2))
    if len(aux2)==1:
        data2.append({''
                     "Tipo":aux2["tipo"][0],"Comprobante":aux2["comprobante"][0],"Concepto":concepto,
                    "Mayor1":aux2["mayor"][0],"Mayor2":"--","codMov":"--",
                    "Movimiento":"--","MontoUSD":round(suma,2),
                    "Detalle":aux2["glosaRenglon"][0],"Glosa":aux2["glosa"][0]
                    })
    else:
        data2.append({''
                     "Tipo":aux2["tipo"][0],"Comprobante":aux2["comprobante"][0],"Concepto":concepto,
                    "Mayor1":aux2["mayor"][0],"Mayor2":aux2["mayor"][1],"codMov":aux2["codMov"][1],
                    "Movimiento":aux2["nomMov"][1],"MontoUSD":round(suma,2),
                    "Detalle":aux2["glosaRenglon"][0],"Glosa":aux2["glosa"][0]
                    })

def debemenoshaber(hasta):
    sumaDebe=0
    sumaHaber=0
    for i in hasta:
        if aux2["mayor"][i] in cuentas and aux2["dh"][i]=='D':
            sumaDebe=sumaDebe+aux2["mn"][i]/6.86
        elif aux2["mayor"][i] in cuentas and aux2["dh"][i]=='H':
            sumaHaber=sumaHaber+aux2["mn"][i]/6.86
    suma=sumaDebe-sumaHaber
    if suma>0:
        dv("Ingreso",suma)
    else:
        dv("Egreso",-suma)

In [13]:
aux1=bd["comprobante"].unique()
consolidado=pd.DataFrame()

for q in range(len(aux1)):
    aux2 = bd[bd["comprobante"]==aux1[q]]
    aux2=aux2.reset_index(drop=True)
    data1=[]
    columnas1=["Tipo","Comprobante","Concepto","Mayor","codMov","Movimiento","MN","Glosa","GlosaDetalle"]
    data2=[]
    columnas2=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
    print("Comprobante ",q)
    print(aux2["comprobante"][0])
    if aux2["tipo"][0]=='V' or aux2["tipo"][0]=='D': ####### los ajustes por arbitraje y revalorizacion ######
        data2=[]
        columnas2=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
        sumaDebe=0
        sumaHaber=0
        for i in aux2.index:
            if aux2["mayor"][i] in cuentas and aux2["dh"][i]=='D':
                sumaDebe=sumaDebe+aux2["mn"][i]/6.86
            elif aux2["mayor"][i] in cuentas and aux2["dh"][i]=='H':
                sumaHaber=sumaHaber+aux2["mn"][i]/6.86
        suma=sumaDebe-sumaHaber
        if suma>0:
            dv("Ingreso",suma)
            #unRegistro=pd.DataFrame(data2,columns=columnas)
            #consolidado=consolidado.append(unRegistro, ignore_index=True)
        else:
            dv("Egreso",-suma)
            #unRegistro=pd.DataFrame(data2,columns=columnas)
            #consolidado=consolidado.append(unRegistro, ignore_index=True)
        unRegistro=pd.DataFrame(data2,columns=columnas2)
        consolidado=consolidado.append(unRegistro, ignore_index=True)
    else: # estos son los casos normales digamos ##############################################################
        #print ("menos Tipo D o V")
        print("Hay duplicado: ",aux2.duplicated(subset=['mn']).any())
        
        if aux2.duplicated(subset=['mn']).any()==True:  ## aqui sacamos los repetidos en la MN
            aux3=aux2.duplicated(subset=['mn'],keep=False)
            aux4=aux3.index[aux3==True].tolist()
            print('aux4')
            print(aux4)
            for x in range(len(aux4)):

                data1.append({
                    "Tipo":aux2["tipo"][aux4[x]],"Comprobante":aux2["comprobante"][aux4[x]],"Concepto":aux2["dh"][aux4[x]],
                    "Mayor":aux2["mayor"][aux4[x]],"codMov":aux2["codMov"][aux4[x]],"Movimiento":aux2["nomMov"][aux4[x]],
                    "MN":aux2["mn"][aux4[x]],"Glosa":aux2["glosa"][aux4[x]],"GlosaDetalle":aux2["glosaRenglon"][aux4[x]],
                            })
                pares=pd.DataFrame(data1,columns=columnas1)

        else:   # Aqui anotamos todos los que no son repetidos
            print('aux2')
            aux2= aux2.sort_values(by='mayor') #, key=cuentas)
            aux2=aux2.reset_index(drop=True)
            print(aux2)
            
            for x in range(len(aux2)):
                #print(aux[x],"=== ",aux2["mn"][aux4[x]])
                data1.append({
                    "Tipo":aux2["tipo"][x],"Comprobante":aux2["comprobante"][x],"Concepto":aux2["dh"][x],
                    "Mayor":aux2["mayor"][x],"codMov":aux2["codMov"][x],"Movimiento":aux2["nomMov"][x],
                    "MN":aux2["mn"][x],"Glosa":aux2["glosa"][x],"GlosaDetalle":aux2["glosaRenglon"][x],
                            })
                pares=pd.DataFrame(data1,columns=columnas1)

            #print(aux2['mayor'][i],' ... ',aux2["comprobante"][i])
            #print('pares ',pares["Comprobante"][0])
        print(pares)
        data2=[]
        columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes

        if len(aux2)==2: ### los pares. Este sería el caso normal
            print("largo de aux2: ",len(aux2))
            if pares["Mayor"][0] in cuentas and pares["Mayor"][1] in cuentas:
                #print("Posible Neteo")
                llenarData("Neteo",0,1)  ######## aqui ver para que el monto sea cero y que en el detalle salga el monto
                unRegistro=pd.DataFrame(data2,columns=columnas2)
            else:     
                if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="D":
                    print('caso 1')
                    llenarData("Ingreso",0,1)        
                if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="H":
                    print('caso 2')
                    llenarData("Egreso",0,1)                   
                if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="D":
                    print('caso 3')
                    llenarData("Ingreso",1,0)                    
                if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="H":            
                    print('caso 4')
                    llenarData("Egreso",1,0)
        else:
            if len(aux2)==1:  ### los sueltitos (se supone que los otros movimientos estan en otras areas, ej. gadm)              
                print("entramos con 1")
                if pares["Concepto"][0]=="D":
                    print('caso 5')
                    llenarData("Ingreso",0,0)
                else:
                    print('caso 6')
                    llenarData("Egreso",0,0)
            else:             # Aqui es cuando no hay duplicados y no son pares ni sueltitos
                print("mas de 2")
                ###
                data2=[]
                columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
                sumaDebe=0
                sumaHaber=0
                for i in range(len(aux2)):
                    if aux2["mayor"][i] in cuentas and aux2["dh"][i]=='D':
                        sumaDebe=sumaDebe+aux2["mn"][i]/6.86
                    elif aux2["mayor"][i] in cuentas and aux2["dh"][i]=='H':
                        sumaHaber=sumaHaber+aux2["mn"][i]/6.86
                suma=sumaDebe-sumaHaber
                if suma>0:
                    dv("Ingreso",suma)
                    #unRegistro=pd.DataFrame(data2,columns=columnas)
                    #consolidado=consolidado.append(unRegistro, ignore_index=True)
                else:
                    dv("Egreso",-suma)
                    #unRegistro=pd.DataFrame(data2,columns=columnas)
                    #consolidado=consolidado.append(unRegistro, ignore_index=True)
                ####

        unRegistro=pd.DataFrame(data2,columns=columnas2)
        consolidado=consolidado.append(unRegistro, ignore_index=True)  

consolidado
#consolidado.to_excel("borrar1.xlsx")

Comprobante  0
2178
Hay duplicado:  True
aux4
[0, 1]
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S         2178        D    485    5782   
1    S         2178        H     10    7266   

                                          Movimiento      MN  \
0  PROVISIONES PARA COMISIONES AL CUSTODIO       ...  646.49   
1  CTA. P99828-AG530 PORTAFOLIO FPA              ...  646.49   

                                               Glosa  GlosaDetalle  
0  ||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...           NaN  
1  ||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...           NaN  
largo de aux2:  2
caso 4
dentro  2178
Comprobante  1
5545
Hay duplicado:  True
aux4
[0, 1]
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S         5545        D    485    5782   
1    S         5545        H     10    7038   

                                          Movimiento       MN  \
0  PROVISIONES PARA COMISIONES AL CUSTODIO       ...  3993.89   
1  CTA. 0728162841 PORTAFOLIO FPAH        

Hay duplicado:  False
aux2
  tipo glosaRenglon dh     mo  moneda  codMov  \
0    S          NaN  D  72.78      34    5896   

                                              nomMov      mn  \
0  NEW YORK - CTA. 3544020682001                 ...  499.27   

                                               glosa  comprobante  mayor  
0  ||RETENCION IUE - BE DEL PORTAFOLIO FPAH POR E...      1025334     10  
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S      1025334        D     10    5896   

                                          Movimiento      MN  \
0  NEW YORK - CTA. 3544020682001                 ...  499.27   

                                               Glosa  GlosaDetalle  
0  ||RETENCION IUE - BE DEL PORTAFOLIO FPAH POR E...           NaN  
entramos con 1
caso 5
dentro  1025334
Comprobante  18
1777343
Hay duplicado:  True
aux4
[0, 1]
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    G      1777343        D     10    5896   
1    G      1777343        H    404    7164 

aux4
[0, 1]
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    O      1996851        D    403    7191   
1    O      1996851        H      7    3491   

                                          Movimiento        MN  \
0  BCO.PYME DE LA COMUNIDAD S.A. CTA.CTE. Y DE EN...  686000.0   
1  DOLARES AMERICANOS EN TESORO PROPIO           ...  686000.0   

                                               Glosa  GlosaDetalle  
0  PAGO CHEQUE: D-19-98365     , BENEFICIARIO: EM...           NaN  
1  PAGO CHEQUE: D-19-98365     , BENEFICIARIO: EM...           NaN  
largo de aux2:  2
caso 4
dentro  1996851
Comprobante  35
1996852
Hay duplicado:  True
aux4
[0, 1]
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    O      1996852        D    403    3907   
1    O      1996852        H      7    3491   

                                          Movimiento          MN  \
0  BCO.MERCANTIL SANTA CRUZ SA - CTA.CTE. Y DE EN...  34300000.0   
1  DOLARES AMERICANOS EN TESORO PROPIO           ...  34300000.0

,Tipo,Comprobante,Concepto,Mayor1,Mayor2,codMov,Movimiento,MontoUSD,Detalle,Glosa
0,S,2178,Egreso,10,485,5782,PROVISIONES PARA COMISIONES AL CUSTODIO ...,94.24,NaN,||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...
1,S,5545,Egreso,10,485,5782,PROVISIONES PARA COMISIONES AL CUSTODIO ...,582.20,NaN,||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...
2,D,21859,Ingreso,22,498,1470,ACREED.DIVERSOS -OPERACIONES CAMBIARIAS ...,0.01,Saldo ME = 93567.36 ;Bs/Mo: 6.86000000000 ;Sal...,AJUSTE COMPLEMENTARIO POR REVALORIZACION SALDO...
3,V,28482,Ingreso,306,17,6296,INTERESES DEVENGADOS ...,7749695.03,Saldo Ant.= -66098.00 ;Bs: 9.59837480000 = $us...,AJUSTE POR ARBITRAJE DE SALDOS DE ACTIVOS DE R...
4,S,1025269,Ingreso,11,606,1535,INTERESES OVERNIGHT CAPITAL TRABAJO ...,2242.85,NaN,'INTERESES DEVENGADOS SOBRE DEPOSITOS'||OVERNI...
5,S,1025272,Ingreso,10,412,3987,CUENTA UNICA DEL TESORO ...,25833.67,A LA CUENTA DIREC.GRAL. AERONAUTICA CIVIL-RECU...,||TRANSFERENCIA DE FONDOS S/G. MENSAJE SWIFT N...
6,S,1025273,Ingreso,10,404,7954,YPFB - EXPORTACION DE UREA Y OTROS ...,2967000.00,POR CUENTA DE SWISS SINGAPORE OVERSEAS ENTERPR...,||TRANSFERENCIA DE FONDOS S/G. MENSAJES SWIFT ...
7,S,1025275,Ingreso,10,404,7954,YPFB - EXPORTACION DE UREA Y OTROS ...,3027000.00,POR CUENTA DE SWISS SINGAPORE OVERSEAS ENTERPR...,||TRANSFERENCIA DE FONDOS S/G. MENSAJES SWIFT ...
8,S,1025283,Ingreso,10,498,8304,ACREEDORES DIVERSOS OPERACIONES DEL SECTOR PUB...,225.00,(CC:H-0046550-2022),'ABONO TRANSITORIO PARA SU POSTERIOR REGULARIZ...
9,S,1025289,Egreso,10,412,3987,CUENTA UNICA DEL TESORO ...,105.11,TRANSFERENCIA DE FONDOS AL JPMORGAN CHASE BANK...,||TRANSFERENCIA DE FOBDOS SEGÚN NOTA MEFP/VTCP...


In [14]:
aux1=bd["comprobante"].unique()
consolidado=pd.DataFrame()

for q in range(len(aux1)):
    aux2 = bd[bd["comprobante"]==aux1[q]]
    aux2=aux2.reset_index(drop=True)
    data1=[]
    columnas1=["Tipo","Comprobante","Concepto","Mayor","codMov","Movimiento","MN","Glosa","GlosaDetalle"]
    data2=[]
    columnas2=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
    print("Comprobante ",q)
    print(aux2["comprobante"][0])
    if aux2["tipo"][0]=='V' or aux2["tipo"][0]=='D': ####### los ajustes por arbitraje y revalorizacion ######
        #data2=[]
        #columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
        ########## RRRRRRRRRRRR
        debemenoshaber(aux2.index)
        ########## RRRRRRRRRRRR   
        unRegistro=pd.DataFrame(data2,columns=columnas2)
        consolidado=consolidado.append(unRegistro, ignore_index=True)
    else: # estos son los casos normales digamos ##############################################################
        
        print("Hay duplicado: ",aux2.duplicated(subset=['mn']).any())
        
        if aux2.duplicated(subset=['mn']).any()==True:  ## aqui sacamos los repetidos en la MN
            aux3=aux2.duplicated(subset=['mn'],keep=False)
            aux4=aux3.index[aux3==True].tolist()
            print('aux4')
            print(aux4)
            if len(aux4)==2:
                for x in range(len(aux4)):

                    data1.append({
                        "Tipo":aux2["tipo"][aux4[x]],"Comprobante":aux2["comprobante"][aux4[x]],"Concepto":aux2["dh"][aux4[x]],
                        "Mayor":aux2["mayor"][aux4[x]],"codMov":aux2["codMov"][aux4[x]],"Movimiento":aux2["nomMov"][aux4[x]],
                        "MN":aux2["mn"][aux4[x]],"Glosa":aux2["glosa"][aux4[x]],"GlosaDetalle":aux2["glosaRenglon"][aux4[x]],
                                })
                    pares=pd.DataFrame(data1,columns=columnas1)
                #################### 1
                print("largo de aux2: ",len(aux2))
                print(pares)
                ###########
                #data2=[]
                #columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
                ##########
                if pares["Mayor"][0] in cuentas and pares["Mayor"][1] in cuentas:
                    #print("Posible Neteo")
                    llenarData("Neteo",0,1)  ######## aqui ver para que el monto sea cero y que en el detalle salga el monto
                    unRegistro=pd.DataFrame(data2,columns=columnas2)
                else:     
                    if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="D":
                        print('caso 1')
                        llenarData("Ingreso",0,1)        
                    if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="H":
                        print('caso 2')
                        llenarData("Egreso",0,1)                   
                    if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="D":
                        print('caso 3')
                        llenarData("Ingreso",1,0)                    
                    if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="H":            
                        print('caso 4')
                        llenarData("Egreso",1,0)
            else:
                
                #@#@#@#@#@#@#
                #data2=[]
                #columnas2=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
               
                ####BBBBBBBBBBBBBBBBBBB
                debemenoshaber(range(len(aux2)))
                ####BBBBBBBBBBBBBBBBBBB
                #@#@#@#@#@#@#
            #################### 1

        else:   # Aqui anotamos todos los que no son repetidos
            print('aux2')
            aux2= aux2.sort_values(by='mayor') #, key=cuentas)
            aux2=aux2.reset_index(drop=True)
            print(aux2)
            
            for x in range(len(aux2)):
                #print(aux[x],"=== ",aux2["mn"][aux4[x]])
                data1.append({
                    "Tipo":aux2["tipo"][x],"Comprobante":aux2["comprobante"][x],"Concepto":aux2["dh"][x],
                    "Mayor":aux2["mayor"][x],"codMov":aux2["codMov"][x],"Movimiento":aux2["nomMov"][x],
                    "MN":aux2["mn"][x],"Glosa":aux2["glosa"][x],"GlosaDetalle":aux2["glosaRenglon"][x],
                            })
                pares=pd.DataFrame(data1,columns=columnas1)
            ################### 2
            print(pares)
            #data2=[]
            #columnas2=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
            
            if len(aux2)==1:  ### los sueltitos (se supone que los otros movimientos estan en otras areas, ej. gadm)              
                print("entramos con 1")
                if pares["Concepto"][0]=="D":
                    print('caso 5')
                    llenarData("Ingreso",0,0)
                else:
                    print('caso 6')
                    llenarData("Egreso",0,0)
            else:             # Aqui es cuando no hay duplicados y no son pares ni sueltitos
                print("mas de 2")
                
                
                #data2=[]
                #columnas2=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
                ######### HHHHHHH
               # if len(aux2)==3 and aux2["mayor"][0] in cuentas and aux2["mayor"][1] in cuentas and aux2["mayor"][2]==680:
               #     llenarData("Neteo",0,1)
               # else:
                debemenoshaber(range(len(aux2)))
               
                ######### HHHHHHH 
            ################### 2
            #print(aux2['mayor'][i],' ... ',aux2["comprobante"][i])
            #print('pares ',pares["Comprobante"][0])
            
        ###$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
        ##### $$$$$$$$$$$$$$$
        unRegistro=pd.DataFrame(data2,columns=columnas2)
        consolidado=consolidado.append(unRegistro, ignore_index=True)  

consolidado

#consolidado.to_excel("borrar10ene.xlsx")

Comprobante  0
2178
Hay duplicado:  True
aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S         2178        D    485    5782   
1    S         2178        H     10    7266   

                                          Movimiento      MN  \
0  PROVISIONES PARA COMISIONES AL CUSTODIO       ...  646.49   
1  CTA. P99828-AG530 PORTAFOLIO FPA              ...  646.49   

                                               Glosa  GlosaDetalle  
0  ||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...           NaN  
1  ||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...           NaN  
caso 4
dentro  2178
Comprobante  1
5545
Hay duplicado:  True
aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S         5545        D    485    5782   
1    S         5545        H     10    7038   

                                          Movimiento       MN  \
0  PROVISIONES PARA COMISIONES AL CUSTODIO       ...  3993.89   
1  CTA. 0728162841 PORTA

1025295
Hay duplicado:  True
aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S      1025295        D     16    6286   
1    S      1025295        H    499    7355   

                                          Movimiento      MN  \
0  TITULOS - PORTAFOLIO DE LIQUIDEZ              ...  266.85   
1  VALORACION TITULOS DE LAS RESERVAS INTERNACION...  266.85   

                                               Glosa  GlosaDetalle  
0  ||REVALORACION DE TITULOS A PRECIOS DE MERCADO...           NaN  
1  ||REVALORACION DE TITULOS A PRECIOS DE MERCADO...           NaN  
caso 1
dentro  1025295
Comprobante  11
1025301
Hay duplicado:  True
aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S      1025301        D     10    5896   
1    S      1025301        H    404    7954   

                                          Movimiento         MN  \
0  NEW YORK - CTA. 3544020682001                 ...  3552519.6   
1  YPFB - EXPORTACION DE 

largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    G      1777620        D     10    5896   
1    G      1777620        H    404    6215   

                                          Movimiento          MN  \
0  NEW YORK - CTA. 3544020682001                 ...  5229980.58   
1  EMPRESA METALURGICA VINTO (EMV)               ...  5229980.58   

                                               Glosa  GlosaDetalle  
0  TRANSFERENCIA DEL EXTERIOR SEGUN SWIFT NO.1732...           NaN  
1  TRANSFERENCIA DEL EXTERIOR SEGUN SWIFT NO.1732...           NaN  
caso 1
dentro  1777620
Comprobante  26
1777623
Hay duplicado:  True
aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    G      1777623        D     10    5896   
1    G      1777623        H    404    6215   

                                          Movimiento          MN  \
0  NEW YORK - CTA. 3544020682001                 ...  8912366.64   
1  EMPRESA METALURGICA VINTO (EMV)               ...

,Tipo,Comprobante,Concepto,Mayor1,Mayor2,codMov,Movimiento,MontoUSD,Detalle,Glosa
0,S,2178,Egreso,10,485,5782,PROVISIONES PARA COMISIONES AL CUSTODIO ...,94.24,NaN,||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...
1,S,5545,Egreso,10,485,5782,PROVISIONES PARA COMISIONES AL CUSTODIO ...,582.20,NaN,||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...
2,D,21859,Ingreso,22,498,1470,ACREED.DIVERSOS -OPERACIONES CAMBIARIAS ...,0.01,Saldo ME = 93567.36 ;Bs/Mo: 6.86000000000 ;Sal...,AJUSTE COMPLEMENTARIO POR REVALORIZACION SALDO...
3,V,28482,Ingreso,306,17,6296,INTERESES DEVENGADOS ...,7749695.03,Saldo Ant.= -66098.00 ;Bs: 9.59837480000 = $us...,AJUSTE POR ARBITRAJE DE SALDOS DE ACTIVOS DE R...
4,S,1025269,Ingreso,11,606,1535,INTERESES OVERNIGHT CAPITAL TRABAJO ...,2242.85,NaN,'INTERESES DEVENGADOS SOBRE DEPOSITOS'||OVERNI...
5,S,1025272,Ingreso,10,412,8467,DIREC.GRAL.AERONAUTICA CIVIL-RECURSOS ESPECIFI...,25833.67,P. CTA. DE IATA INTERNATIONAL AIR.,||TRANSFERENCIA DE FONDOS S/G. MENSAJE SWIFT N...
6,S,1025273,Ingreso,10,404,7954,YPFB - EXPORTACION DE UREA Y OTROS ...,2967000.00,POR CUENTA DE SWISS SINGAPORE OVERSEAS ENTERPR...,||TRANSFERENCIA DE FONDOS S/G. MENSAJES SWIFT ...
7,S,1025275,Ingreso,10,404,7954,YPFB - EXPORTACION DE UREA Y OTROS ...,3027000.00,POR CUENTA DE SWISS SINGAPORE OVERSEAS ENTERPR...,||TRANSFERENCIA DE FONDOS S/G. MENSAJES SWIFT ...
8,S,1025283,Ingreso,10,498,8304,ACREEDORES DIVERSOS OPERACIONES DEL SECTOR PUB...,225.00,(CC:H-0046550-2022),'ABONO TRANSITORIO PARA SU POSTERIOR REGULARIZ...
9,S,1025289,Egreso,10,412,3987,CUENTA UNICA DEL TESORO ...,105.11,TRANSFERENCIA DE FONDOS AL JPMORGAN CHASE BANK...,||TRANSFERENCIA DE FOBDOS SEGÚN NOTA MEFP/VTCP...


In [15]:
texto1 = "PAGO A EXIMBANK CHINA POPUL PRÉSTAMO PBC 2015 (08) 350 VCTO. 21-01-2022 POR CUENTA DE TGN , NTI. 015626 VALOR 21-01-2022 CAPITAL USD 27.355.555,56 COMISIONES USD 260.358,75"
texto2= "PAGO A AFD PRÉSTAMO CBO-1009-01-J VCTO. 21-01-2022 POR CUENTA DE ENDE , VALOR 21-01-2022  INTERESES EUR 168.666,67"

#capital = re.findall(" CAPITAL (.*)(?= [IC])" , texto1)
capital = re.findall(" CAPITAL (.*)(?= [INTERESES][COMISIONES])" , texto1)
intereses = re.findall(" INTERESES (.*)(?=[C\s])" , texto1)
comisiones = re.findall(" COMISIONES (.*)" , texto1)

#t= pd.DataFrame(search)
#t
print(capital, intereses, comisiones)

[] [] ['USD 260.358,75']


In [16]:
aux1=bd["comprobante"].unique()
consolidado=pd.DataFrame()

for q in range(len(aux1)):
    aux2 = bd[bd["comprobante"]==aux1[q]]
    aux2=aux2.reset_index(drop=True)
    data1=[]
    columnas1=["Tipo","Comprobante","Concepto","Mayor","codMov","Movimiento","MN","Glosa","GlosaDetalle"]
    data2=[]
    columnas2=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
    print("Comprobante ",q)
    print(aux2["comprobante"][0])
    if aux2["tipo"][0]=='V' or aux2["tipo"][0]=='D': ####### los ajustes por arbitraje y revalorizacion ######
        debemenoshaber(aux2.index)
        unRegistro=pd.DataFrame(data2,columns=columnas2)
        consolidado=consolidado.append(unRegistro, ignore_index=True)
    else: # estos son los casos normales digamos ##############################################################
        print("Hay duplicado: ",aux2.duplicated(subset=['mn']).any())
        if aux2.duplicated(subset=['mn']).any()==True:  ## aqui sacamos los repetidos en la MN
            aux3=aux2.duplicated(subset=['mn'],keep=False)
            aux4=aux3.index[aux3==True].tolist()
            print('aux4')
            print(aux4)
            if len(aux4)==2:
                for x in range(len(aux4)):
                    data1.append({
                        "Tipo":aux2["tipo"][aux4[x]],"Comprobante":aux2["comprobante"][aux4[x]],"Concepto":aux2["dh"][aux4[x]],
                        "Mayor":aux2["mayor"][aux4[x]],"codMov":aux2["codMov"][aux4[x]],"Movimiento":aux2["nomMov"][aux4[x]],
                        "MN":aux2["mn"][aux4[x]],"Glosa":aux2["glosa"][aux4[x]],"GlosaDetalle":aux2["glosaRenglon"][aux4[x]],
                                })
                    pares=pd.DataFrame(data1,columns=columnas1)
                print("largo de aux2: ",len(aux2))
                print(pares)
                if pares["Mayor"][0] in cuentas and pares["Mayor"][1] in cuentas:
                    #print("Posible Neteo")
                    llenarData("Neteo",0,1)  ######## aqui ver para que el monto sea cero y que en el detalle salga el monto
                    unRegistro=pd.DataFrame(data2,columns=columnas2)
                else:     
                    if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="D":
                        print('caso 1')
                        llenarData("Ingreso",0,1)        
                    if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="H":
                        print('caso 2')
                        llenarData("Egreso",0,1)                   
                    if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="D":
                        print('caso 3')
                        llenarData("Ingreso",1,0)                    
                    if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="H":            
                        print('caso 4')
                        llenarData("Egreso",1,0)
            else:
                debemenoshaber(range(len(aux2)))
        else:   # Aqui anotamos todos los que no son repetidos
            print('aux2')
            aux2= aux2.sort_values(by='mayor') #, key=cuentas)
            aux2=aux2.reset_index(drop=True)
            print(aux2)
            for x in range(len(aux2)):
                data1.append({
                    "Tipo":aux2["tipo"][x],"Comprobante":aux2["comprobante"][x],"Concepto":aux2["dh"][x],
                    "Mayor":aux2["mayor"][x],"codMov":aux2["codMov"][x],"Movimiento":aux2["nomMov"][x],
                    "MN":aux2["mn"][x],"Glosa":aux2["glosa"][x],"GlosaDetalle":aux2["glosaRenglon"][x],
                            })
                pares=pd.DataFrame(data1,columns=columnas1)
            print(pares)
            if len(aux2)==1:  ### los sueltitos (se supone que los otros movimientos estan en otras areas, ej. gadm)              
                print("entramos con 1")
                if pares["Concepto"][0]=="D":
                    print('caso 5')
                    llenarData("Ingreso",0,0)
                else:
                    print('caso 6')
                    llenarData("Egreso",0,0)
            else:             # Aqui es cuando no hay duplicados y no son pares ni sueltitos
                print("mas de 2")
                debemenoshaber(range(len(aux2)))
        unRegistro=pd.DataFrame(data2,columns=columnas2)
        consolidado=consolidado.append(unRegistro, ignore_index=True)  

consolidado
#consolidado.to_excel("borrar10ene.xlsx")

Comprobante  0
2178
Hay duplicado:  True
aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S         2178        D    485    5782   
1    S         2178        H     10    7266   

                                          Movimiento      MN  \
0  PROVISIONES PARA COMISIONES AL CUSTODIO       ...  646.49   
1  CTA. P99828-AG530 PORTAFOLIO FPA              ...  646.49   

                                               Glosa  GlosaDetalle  
0  ||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...           NaN  
1  ||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...           NaN  
caso 4
dentro  2178
Comprobante  1
5545
Hay duplicado:  True
aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S         5545        D    485    5782   
1    S         5545        H     10    7038   

                                          Movimiento       MN  \
0  PROVISIONES PARA COMISIONES AL CUSTODIO       ...  3993.89   
1  CTA. 0728162841 PORTA

aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S      1025301        D     10    5896   
1    S      1025301        H    404    7954   

                                          Movimiento         MN  \
0  NEW YORK - CTA. 3544020682001                 ...  3552519.6   
1  YPFB - EXPORTACION DE UREA Y OTROS            ...  3552519.6   

                                               Glosa  \
0  ||TRANSFERENCIA DE FONDOS S/G. MENSAJES SWIFT ...   
1  ||TRANSFERENCIA DE FONDOS S/G. MENSAJES SWIFT ...   

                                      GlosaDetalle  
0  A LA CUENTA YPFB - EXPORTACION DE UREA Y OTROS.  
1            POR CUENTA DE HINOVE AGROCIENCIA S.A.  
caso 1
dentro  1025301
Comprobante  12
1025312
Hay duplicado:  True
aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S      1025312        D     10    5896   
1    S      1025312        H    414    7866   

                                          Movimiento      

Comprobante  21
1777480
Hay duplicado:  True
aux4
[0, 4]
largo de aux2:  6
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    G      1777480        D    498    6818   
1    G      1777480        H     10    5896   

                                          Movimiento      MN  \
0  DEPOSITOS SECTOR PUBLICO - OP. CAMBIARIAS     ...  3292.8   
1  NEW YORK - CTA. 3544020682001                 ...  3292.8   

                                               Glosa  \
0  TRANSFERENCIA DE FONDOS AL EXTERIOR A SOLICITU...   
1  TRANSFERENCIA DE FONDOS AL EXTERIOR A SOLICITU...   

                                        GlosaDetalle  
0                                                NaN  
1  HECTOR FERNANDO SIAS MONTESINOS, CTA. 68600150...  
caso 4
dentro  1777480
Comprobante  22
1777481
Hay duplicado:  False
aux2
  tipo                                       glosaRenglon dh          mo  \
0    G  TERMINAL PUERTO ARICA S.A., CTA. 005100026386 ...  H   152675.37   
1    G        LIB. 00594012001

Hay duplicado:  True
aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    G      1777626        D     10    5896   
1    G      1777626        H    404    6215   

                                          Movimiento           MN  \
0  NEW YORK - CTA. 3544020682001                 ...  10623426.53   
1  EMPRESA METALURGICA VINTO (EMV)               ...  10623426.53   

                                               Glosa  GlosaDetalle  
0  TRANSFERENCIA DEL EXTERIOR SEGUN SWIFT NO.1739...           NaN  
1  TRANSFERENCIA DEL EXTERIOR SEGUN SWIFT NO.1739...           NaN  
caso 1
dentro  1777626
Comprobante  28
1777629
Hay duplicado:  True
aux4
[0, 1]
largo de aux2:  2
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    G      1777629        D     10    5896   
1    G      1777629        H    412    5970   

                                          Movimiento         MN  \
0  NEW YORK - CTA. 3544020682001                 ...  2144436.0   
1  CUENTA UNICA DE

,Tipo,Comprobante,Concepto,Mayor1,Mayor2,codMov,Movimiento,MontoUSD,Detalle,Glosa
0,S,2178,Egreso,10,485,5782,PROVISIONES PARA COMISIONES AL CUSTODIO ...,94.24,NaN,||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...
1,S,5545,Egreso,10,485,5782,PROVISIONES PARA COMISIONES AL CUSTODIO ...,582.20,NaN,||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...
2,D,21859,Ingreso,22,498,1470,ACREED.DIVERSOS -OPERACIONES CAMBIARIAS ...,0.01,Saldo ME = 93567.36 ;Bs/Mo: 6.86000000000 ;Sal...,AJUSTE COMPLEMENTARIO POR REVALORIZACION SALDO...
3,V,28482,Ingreso,306,17,6296,INTERESES DEVENGADOS ...,7749695.03,Saldo Ant.= -66098.00 ;Bs: 9.59837480000 = $us...,AJUSTE POR ARBITRAJE DE SALDOS DE ACTIVOS DE R...
4,S,1025269,Ingreso,11,606,1535,INTERESES OVERNIGHT CAPITAL TRABAJO ...,2242.85,NaN,'INTERESES DEVENGADOS SOBRE DEPOSITOS'||OVERNI...
5,S,1025272,Ingreso,10,412,8467,DIREC.GRAL.AERONAUTICA CIVIL-RECURSOS ESPECIFI...,25833.67,P. CTA. DE IATA INTERNATIONAL AIR.,||TRANSFERENCIA DE FONDOS S/G. MENSAJE SWIFT N...
6,S,1025273,Ingreso,10,404,7954,YPFB - EXPORTACION DE UREA Y OTROS ...,2967000.00,POR CUENTA DE SWISS SINGAPORE OVERSEAS ENTERPR...,||TRANSFERENCIA DE FONDOS S/G. MENSAJES SWIFT ...
7,S,1025275,Ingreso,10,404,7954,YPFB - EXPORTACION DE UREA Y OTROS ...,3027000.00,POR CUENTA DE SWISS SINGAPORE OVERSEAS ENTERPR...,||TRANSFERENCIA DE FONDOS S/G. MENSAJES SWIFT ...
8,S,1025283,Ingreso,10,498,8304,ACREEDORES DIVERSOS OPERACIONES DEL SECTOR PUB...,225.00,(CC:H-0046550-2022),'ABONO TRANSITORIO PARA SU POSTERIOR REGULARIZ...
9,S,1025289,Egreso,10,412,3987,CUENTA UNICA DEL TESORO ...,105.11,TRANSFERENCIA DE FONDOS AL JPMORGAN CHASE BANK...,||TRANSFERENCIA DE FOBDOS SEGÚN NOTA MEFP/VTCP...


In [17]:
movimientosAux= movimientos.drop(columns=['nomMovimiento'])
conCodigos = pd.merge(consolidado,movimientosAux, on=['Concepto', 'Mayor2','codMov'], how='left')
conCodigos

######## el 01feb tiene buenos moviemientos para ver glosas

,Tipo,Comprobante,Concepto,Mayor1,Mayor2,codMov,Movimiento,MontoUSD,Detalle,Glosa,codigoBC,codigoPropio
0,S,2178,Egreso,10,485,5782,PROVISIONES PARA COMISIONES AL CUSTODIO ...,94.24,NaN,||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...,NaN,NaN
1,S,5545,Egreso,10,485,5782,PROVISIONES PARA COMISIONES AL CUSTODIO ...,582.20,NaN,||REGISTRO DEL PAGO DE COMISION AL CUSTODIO DE...,NaN,NaN
2,D,21859,Ingreso,22,498,1470,ACREED.DIVERSOS -OPERACIONES CAMBIARIAS ...,0.01,Saldo ME = 93567.36 ;Bs/Mo: 6.86000000000 ;Sal...,AJUSTE COMPLEMENTARIO POR REVALORIZACION SALDO...,NaN,NaN
3,V,28482,Ingreso,306,17,6296,INTERESES DEVENGADOS ...,7749695.03,Saldo Ant.= -66098.00 ;Bs: 9.59837480000 = $us...,AJUSTE POR ARBITRAJE DE SALDOS DE ACTIVOS DE R...,NaN,NaN
4,S,1025269,Ingreso,11,606,1535,INTERESES OVERNIGHT CAPITAL TRABAJO ...,2242.85,NaN,'INTERESES DEVENGADOS SOBRE DEPOSITOS'||OVERNI...,RENTA (BCI14),Intereses Overnight
5,S,1025272,Ingreso,10,412,8467,DIREC.GRAL.AERONAUTICA CIVIL-RECURSOS ESPECIFI...,25833.67,P. CTA. DE IATA INTERNATIONAL AIR.,||TRANSFERENCIA DE FONDOS S/G. MENSAJE SWIFT N...,CORRIENTES (BCI12),Dir. Gral. Aeronáutica Civil
6,S,1025273,Ingreso,10,404,7954,YPFB - EXPORTACION DE UREA Y OTROS ...,2967000.00,POR CUENTA DE SWISS SINGAPORE OVERSEAS ENTERPR...,||TRANSFERENCIA DE FONDOS S/G. MENSAJES SWIFT ...,Y.P.F.B. Otros (BCI5),YPFB UREA
7,S,1025275,Ingreso,10,404,7954,YPFB - EXPORTACION DE UREA Y OTROS ...,3027000.00,POR CUENTA DE SWISS SINGAPORE OVERSEAS ENTERPR...,||TRANSFERENCIA DE FONDOS S/G. MENSAJES SWIFT ...,Y.P.F.B. Otros (BCI5),YPFB UREA
8,S,1025283,Ingreso,10,498,8304,ACREEDORES DIVERSOS OPERACIONES DEL SECTOR PUB...,225.00,(CC:H-0046550-2022),'ABONO TRANSITORIO PARA SU POSTERIOR REGULARIZ...,NaN,NaN
9,S,1025289,Egreso,10,412,3987,CUENTA UNICA DEL TESORO ...,105.11,TRANSFERENCIA DE FONDOS AL JPMORGAN CHASE BANK...,||TRANSFERENCIA DE FOBDOS SEGÚN NOTA MEFP/VTCP...,Ver Glosa,Ver Glosa
